In [ ]:
import socket
hostname = socket.gethostname()

if hostname.startswith('s-sc'):
    !squeue --job $SLURM_JOB_ID 
    !squeue -h -j $SLURM_JOBID -o %L
elif hostname.startswith('jlab-pc13'):
    %env CUDA_VISIBLE_DEVICES=1

In [ ]:
import wholebrain as wb
import opm_img.mini as opm

import numpy as np
import cupy as cp
import cuml
import rmm
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
import importlib
importlib.reload(wb)
importlib.reload(wb.visualize)
importlib.reload(wb.cluster)
importlib.reload(wb.regression)
importlib.reload(wb.traces)

## files and paths

In [ ]:
#dataset_name = '20221123_1045_No_stimulus'
#dataset_name = '20221201_1812_actb2-nucG8mPilot'
#dataset_name = '20211105_1332_no_stimulus'
#dataset_name = '20221116_2043_stimset_2022-LR-0'
dataset_name = '20210726_1244_stimset_#2021-1c'

pn_in = '~/hpc-r/opm2021/recordings/'
pn_out = '~/hpc-rw/deleteme-benjamin/'
url_home = '/user/benju/files'

paths = opm.io.Resolver(dataset_name = dataset_name, 
                        pn_in = pn_in, 
                        pn_out = pn_out,
                        url_home = url_home)

## load and select traces

In [ ]:
#load traces
tracing_result = opm.io.load_from_h5(paths.traces)

In [ ]:
# inspect trace quality and filter
non_zero_traces = ((tracing_result['traces']==0).sum(0) == 0)
cov_vals = np.sort(tracing_result['cov_vals'][non_zero_traces])
plt.figure()
plt.plot(cov_vals, np.r_[cov_vals.size:0:-1])
plt.gca().set(xlim=(1,1e5), xlabel='covariance value', ylabel='# cells included')
plt.xscale('log')
plt.tight_layout()

In [ ]:
#manually set treshold based on graph above
cov_thresh = 100

traces_selector = (tracing_result['cov_vals'] > cov_thresh) & non_zero_traces
selected_traces = tracing_result['traces'][:,traces_selector]
selected_traces.shape

## classical dff calculation

In [ ]:
traces_dff = wb.traces.getdff.gpu(selected_traces[240:])

In [ ]:
cix, fig = wb.visualize.rastermap.gpu(traces_dff, embed_options=dict(n_clusters=500), vmin=0, vmax=1, cmap='gray')

## deconvolution of raw traces

In [ ]:
%%time
traces_dec, polyfit, loss, T = wb.traces.simple_sparse_deconvolve.gpu(selected_traces[240:], alpha=.1, tau=4, fit_poly=2, force_positive=True, n_iter=500)

In [ ]:
traces_rec = T @ np.vstack([traces_dec, polyfit])
traces_F0 = T[:,-polyfit.shape[0]:] @ polyfit

In [ ]:
plt.figure(figsize=(16,12))
plt.subplot(2,2,1)
plt.imshow(selected_traces[240:].T[cix], aspect='auto', vmin=0, vmax=1000, cmap='gray')
plt.gca().set_title('raw traces')
plt.subplot(2,2,2)
plt.imshow(selected_traces[240:].T[cix] / traces_F0.T[cix] - 1, aspect='auto', vmin=0, vmax=1, cmap='gray')
plt.gca().set_title('dF/F0_poly  (scaled 0..100%)')
plt.subplot(2,2,3)
plt.gca().set_title('deconvolved')
plt.imshow(traces_dec.T[cix], aspect='auto', vmin=0, vmax=100, cmap='gray')
plt.subplot(2,2,4)
plt.gca().set_title('deconvolved / F0_poly (scaled 0..20%)')
plt.imshow(traces_dec.T[cix]/traces_F0.T[cix], aspect='auto', vmin=0, vmax=.2, cmap='gray')

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
plt.gca().set_title('raw traces and Lassso fit')
#plt.plot(traces_rec.T[cix][::300].T);
#plt.plot(selected_traces[240:].T[cix][::300].T, alpha=0.1);
wb.visualize.plot_waterfall(traces_rec.T[cix][16::700].T, sep=1000, linewidth=0.5)
wb.visualize.plot_waterfall(selected_traces[240:].T[cix][16::700].T, sep=1000, alpha=0.5, linewidth=1)
plt.subplot(2,1,2)
plt.gca().set_title('deconvolved')
wb.visualize.plot_waterfall(traces_dec.T[cix][16::700].T, sep=1000, linewidth=0.5)

## Playground